In [40]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
import torch
import pickle

%load_ext autoreload
%autoreload 2
import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [43]:
test_dataset = datasets.FashionMNIST(
    root='./data_mnist_test',
    train=False,
    download=True
)

In [44]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [45]:
from ResNet.model.residual_attention_network import ResidualAttentionModel_92_32input_update as ResidualAttentionModel

model_res = ResidualAttentionModel().to(device)
model_res.load_state_dict(torch.load('./ResNet/weights/ResidualAttentionModel_92_32input_update_2025-06-16_23:37:33.pth', weights_only=True))
model_res.eval()

transform_resnet = transforms.Compose([
    transforms.Pad(padding=2),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
])

In [46]:
path = "./ResNet/results/ResidualAttentionModel_92_32input_update_2025-06-16_23:37:33/history.pickle"
with open(path, 'rb') as f:
    data = pickle.load(f)

In [47]:
def list_to_dictionary(data):
    keys_ = data[0].keys()
    length_ = len(data)

    temp_ = {}
    for key_ in keys_:
        temp_[key_] = []
        for i in range(length_):
            temp_[key_].append(data[i][key_])

    return temp_

In [48]:
data = list_to_dictionary(data)

In [49]:
print(data.keys())

dict_keys(['loss', 'val_loss', 'train_acc', 'val_acc', 'epoch_time'])


In [50]:
data_num = 32

x = [test_dataset[i][0] for i in range(16)]  # PIL Images
y = [test_dataset[i][1] for i in range(16)]

In [51]:
compare_dict = {
    'ResNet': {'model': model_res, 'transform': transform_resnet, 'history': data},
}

In [52]:
import importlib
import metrics
importlib.reload(metrics)
metrics.compare_models(compare_dict, x, y, device)

--- Rozpoczynanie analizy modeli ---
Analizowanie: ResNet...
(16,)
(16,)
⚠️ Ostrzeżenie: Nie udało się obliczyć FLOPS. Błąd: 'ResidualAttentionModel_92_32input_update' object has no attribute 'inputs'


KeyError: "['AUC (micro)'] not in index"